# 0 - Environment Set Up 

In [1]:
# Install required packages
!pip install -r ../requirements.txt

# 1 - Project Set Up

## 1.1 - Map to Root

In [2]:
REL_PATH_TO_ROOT = "../"

In [3]:
import sys
import os
import json
import pandas as pd

In [4]:
sys.path.insert(0,REL_PATH_TO_ROOT)

In [5]:
from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

In [6]:
test_root_dir(REL_PATH_TO_ROOT)

Root set up correctly


## 1.2 - Set-Up Imports

In [7]:
from prompt_manager.manager import PromptManager
from prompt_manager.fetcher import fetch_prompt
from src.api import generate_outputs_openai

ModuleNotFoundError: No module named 'openai'

## 1.3 - Load Hallucination Dataset

In [17]:
input_path = "../data/hallucination_qa_data.json"

with open(input_path, 'r', encoding="utf-8") as f:
    data = []
    for line in f:
        data.append(json.loads(line))

    hallucination_df = pd.DataFrame(data)
            

In [18]:
hallucination_df.shape

(10000, 4)

In [19]:
hallucination_df.head()

,knowledge,question,right_answer,hallucinated_answer
0,Arthur's Magazine (1844–1846) was an American ...,Which magazine was started first Arthur's Maga...,Arthur's Magazine,First for Women was started first.
1,The Oberoi family is an Indian family that is ...,The Oberoi family is part of a hotel company t...,Delhi,The Oberoi family's hotel company is based in ...
2,"Allison Beth ""Allie"" Goertz (born March 2, 199...",Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"Allie Goertz wrote a song about Milhouse, a po..."
3,"Margaret ""Peggy"" Seeger (born June 17, 1935) i...",What nationality was James Henry Miller's wife?,American,James Henry Miller's wife was British.
4,It is a hygroscopic solid that is highly solu...,Cadmium Chloride is slightly soluble in this c...,alcohol,water with a hint of alcohol


## 1.4 - Sample from the Hallucination Dataset

The original dataset contains 10K records. To lower the likelihood of hitting rate limit, we need to created a random sample from the original data. Sugguested size: 100-150 records.

# 2 - Run Your Evaluation 

## 2.1 - Get Your Prompts

In [16]:
SEQUENCE = ["task_1","hallucination_detector"]

prompt_template = fetch_prompt(SEQUENCE,use_latest_version=True)

print(prompt_template)

Here's the context:
    {CONTEXT}

    Here's what the AI said:

    {RESPONSE}

    Was the response a hallucination?
    
    Say 'True' if it's hallucination and 'False' if it's not a hallucination and nothing else!


In [ ]:
evaluator_responses = []

# Loop through dataset
for _, row in hal_df.iterrows():

    # Get inputs and place into dictionary format
    context = row["knowledge"]

    response = row["chatbot_response"]

    row_inputs = {"CONTEXT" : context, "RESPONSE" : response}

    # Initialise prompt to validate and format inputs
    prompt = PromptManager(template=prompt_template,inputs=row_inputs)
    prompt.validate_inputs()
    prompt.format_inputs()

    # Send prompt and collect response
    response = generate_outputs_openai(prompt.prompt)
    evaluator_responses.append(response)

hallucination_df["evaluator_response"] = evaluator_responses
display(hallucination_df.head(5))